# 加载基本库

In [1]:
import collections
import pandas as pd
import datetime
import time
import math
from gensim.models import Word2Vec
# from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
from sklearn.metrics import log_loss
import xgboost
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
%run FeatureProcess.py
# from fastFM.datasets import make_user_item_regression
# from fastFM import als
from sklearn.metrics import mean_squared_error
import scipy.sparse as sp
import numpy as np
from sklearn import preprocessing
sklearn.__version__

'0.19.1'

# 加载数据项

In [2]:
train_df_org = pd.read_table('/Users/yuhua/先验CTR模型线上观察/Alimama比赛/round1_ijcai_18_train_20180301.txt',sep=' ')
test_df_org = pd.read_table('/Users/yuhua/先验CTR模型线上观察/Alimama比赛/round1_ijcai_18_test_a_20180301.txt',sep=' ')



In [3]:
train_df_org.columns

train_df = train_df_org.copy()#.set_index("context_id")
test_df = test_df_org.copy()#.set_index("context_id")

test_df["is_trade"] = -1
train_df_prev = train_df.append(test_df)
train_df_prev = train_df_prev.reset_index()
# train_df_prev = pd.concat([train_df, test_df], axis=0)


# 预处理数据

In [4]:
# 时间处理: 分离天, 星期几, 上中下午/晚上, 小时数
# date最终不使用，直接用day(第 0 - 7 天)

from datetime import datetime

def extract_date(x):
    d = datetime.fromtimestamp(x)
    return d.strftime("%Y-%m-%d")
def extract_time(x):
    d = datetime.fromtimestamp(x)
    return d.strftime("%Y-%m-%d %H:%M:%S")
def extract_weekday(x):
    d = datetime.fromtimestamp(x)
    return d.weekday()
def extract_hour(x):
    d = datetime.fromtimestamp(x)
    return d.hour

train_df_prev['time'] = train_df_prev['context_timestamp'].apply(lambda x: extract_time(x))
train_df_prev['date'] = train_df_prev['context_timestamp'].apply(lambda x: extract_date(x))
train_df_prev['weekday'] = train_df_prev['context_timestamp'].apply(lambda x: extract_weekday(x))
train_df_prev['hour'] = train_df_prev['context_timestamp'].apply(lambda x: extract_hour(x))

m = {}
for idx, date in enumerate(sorted(train_df_prev['date'].unique())):
    m[date]=idx
print(m)

for idx, d in enumerate(sorted(train_df_prev['date'].unique())):
    train_df_prev["day"] = train_df_prev['date'].map(lambda x: m[x])

{'2018-09-24': 6, '2018-09-25': 7, '2018-09-20': 2, '2018-09-21': 3, '2018-09-22': 4, '2018-09-23': 5, '2018-09-19': 1, '2018-09-18': 0}


# 拆解复杂类型

In [5]:
from numpy import *
#简化list等复杂类型的结构
#item_category_list全展开
#item_property_list全展开取频率>0.05的数据
#predict_category_property会计算跟item prop和cate的余弦相似度
#具体是把两个list的数据拼成cate:-1和cate:prop两种方式拼成一个字符串再跟predict_category_property的数据计算相似度
def get_icl_map(df):
    print("get_icl_map ... ")
    dfX = df.copy()
    dfX = dfX['item_category_list'].str.split(';', expand=True)

#     s = sorted(list(dfX[0].unique()) + list(dfX[1].unique()) + list(dfX[2].unique()))
    m = {}
    for i in dfX[0].unique():
        if i == None:
            continue
        m[i] = "1"
    
    for i in dfX[1].unique():
        if i == None:
            continue
        m[i] = "2"
    
    for i in dfX[2].unique():
        if i == None:
            continue
        m[i] = "3"
    return m

def get_ipl_map(df):
    print("get_ipl_map ... ")
    df1 = df.copy()
    dfX = df1.copy()['item_property_list'].str.split(';')
    dfX = pd.DataFrame(dfX)
    
    m = collections.defaultdict(float)
    idx = 0
    for _, row in dfX.iterrows():
        for i in row[0]:
            m[i] += 1
    
    ll = len(dfX)
    for k,v in m.items():
        m[k] = v / ll
    return m

def cos_sim(a,b):
    if type(a) == type(b) == list:
        am,bm = {},{}
        for i in a:
            am[i] = 1
        for i in b:
            bm[i] = 1
        
        a = am
        b = bm
    
    up = 0.
    down = 0.
    for k in a:
        if k in b:
            up += (a[k]*b[k])
    down = math.sqrt(len(a))*math.sqrt(len(b))
    
    return up/down

class mfilter:
    def __init__(self, m):
        self.m = m
    def __call__(self, x):
        ret = []
        for i in x:
            if i in self.m:
                ret.append(i)
        return ret

class prop_layer_cnt:
    def __init__(self, m):
        for k, v in m.items():
            m[k].append(v[1]/v[0])
        self.m = m
        
#     def __call__(self, x):
#         ret = [0,0,0,0,0,0]
#         for i in x:
#             v = self.m[i]
#             if v[0]>5000:
#                 if v[2]>0.025:
#                     ret[0] += 1
#                 if v[2]<0.01:
#                     ret[1] += 1
#             if 2000<v[0]<=5000:
#                 if v[2]>0.025:
#                     ret[2] += 1
#                 if v[2]<0.01:
#                     ret[3] += 1
#             if 500<v[0]<=2000:
#                 if v[2]>0.025:
#                     ret[4] += 1
#                 if v[2]<0.01:
#                     ret[5] += 1
    def __call__(self, x):
        ret = [0]*20
        for i in x:
            if i not in self.m:
                continue
            v = self.m[i]
            if v[0]>5000:
                if v[2] > 0.04:
                    ret[0] += 1
                if 0.04 >= v[2] > 0.03:
                    ret[1] += 1
                if 0.03 >= v[2] > 0.02:
                    ret[2] += 1
                if 0.02 >= v[2] > 0.01:
                    ret[3] += 1
                if v[2] <= 0.01:
                    ret[4] += 1
            if 2000<v[0]<=5000:
                if v[2] > 0.04:
                    ret[5] += 1
                if 0.04 >= v[2] > 0.03:
                    ret[6] += 1
                if 0.03 >= v[2] > 0.02:
                    ret[7] += 1
                if 0.02 >= v[2] > 0.01:
                    ret[8] += 1
                if v[2] <= 0.01:
                    ret[9] += 1
            if 500<v[0]<=2000:
                if v[2] > 0.04:
                    ret[10] += 1
                if 0.04 >= v[2] > 0.03:
                    ret[11] += 1
                if 0.03 >= v[2] > 0.02:
                    ret[12] += 1
                if 0.02 >= v[2] > 0.01:
                    ret[13] += 1
                if v[2] <= 0.01:
                    ret[14] += 1
            if v[0]<=500:
                if v[2] > 0.04:
                    ret[15] += 1
                if 0.04 >= v[2] > 0.03:
                    ret[16] += 1
                if 0.03 >= v[2] > 0.02:
                    ret[17] += 1
                if 0.02 >= v[2] > 0.01:
                    ret[18] += 1
                if v[2] <= 0.01:
                    ret[19] += 1
                
        return ret
    
class stat_prop_trade_rate:
    def __init__(self):
        self.m = {}
    def __call__(self, x):
        m = self.m
        is_trade = (1 == x["is_trade"])
        for prop in x["item_property_list"]:
            prop = prop.strip()
            if m.get(prop) == None:
                m[prop] = [0.,0.]
            m[prop][0] += 1
            if is_trade:
                m[prop][1] += 1
    
    def get_m(self):
        import copy as cp
        return cp.deepcopy(self.m)
    
    def get_topbutton_cnt(self, top, button, filt):
        m = self.m
        ma = {}
        for k, v in m.items():
            if v[0] > filt:
                ma[k] = v[1]/v[0]

        a = sorted(ma.items(), lambda x, y: cmp(x[1], y[1]))

        ret = {}
        for i in a[:button]:
            ret[i[0]] = ma[i[0]]

        for i in a[-top:]:
            ret[i[0]] = ma[i[0]]

        return ret
    
    def get_topbutton_cnt_filter(self, top, button, filt):
        m = self.get_topbutton_cnt(top, button, filt)
        return mfilter(m)

class prop2vec:
    def __init__(self, size, d):
        self.model = None
        try:
            self.model = Word2Vec.load("p2v.model")
            print("loaded from cache!")
        except:
            pass
        if self.model == None:
            self.model = Word2Vec(d, size=size, min_count=1)
            self.model.save("p2v.model")
    def __call__(self, x):
        s = self.model[x[0]]
        for idx, i in enumerate(x):
            if idx == 0:
                continue
            s = s+self.model[i]
        return list(s)
            
def process_complex_types(dfX, icl_map, ipl_map):
    def filter_unless_cate(arr):
        ret = []
        for i in arr:
            if i in icl_map:
                ret.append(i)
        if len(ret) == 0:
            return None
        else:
            return ret
    
    def filter_unless_prop(arr):
        ret = []
        for i in arr:
            freq = ipl_map.get(i, 0.)
            if freq > 0.05:
                ret.append(i)
            else:
                ret.append(1)
        if len(ret) == 0:
            return None
        else:
            return unique_list(ret)
    
    def unique_list(arr):
        return list(set(arr))
    
    def filter_bad_data(arr):
        new_arr = []
        for i in arr:
            i = i.strip()
            if i == "-1":
                continue
            new_arr.append(i)
        return unique_list(new_arr)
    
    #{cate}:-1命中则为1分
    #{cate}:{prop}命中则为2分
    #后期优化权重
    def inner_product_recall_items(line):
        line = line.split("|")
        item_category_list = unique_list(line[0].split(";"))
        item_property_list = unique_list(line[1].split(";"))
        
        #删掉-1
        if "-1" in item_property_list:
            del item_property_list[item_property_list.index("-1")]
        
        
        #抽出预测的cate_list和prop_list
        line[2] = line[2].split(";")
        
        pitem_category_list_prop = {}
        pitem_category_list = []
        pitem_property_list = []
        for l in line[2]:
            l = l.split(":")
            
            if l[0] != -1:
                pitem_category_list.append(l[0])
            if len(l) >= 2 and l[1] != -1:
                l[1] = l[1].split(",")
                pitem_property_list.append(l[1])
                pitem_category_list_prop[l[0]] = l[1]
            if len(l) >= 3:
                print "FUCK?"

        
        #计算预测的cate相似度+prop相似度
        csim = cos_sim(item_category_list, pitem_category_list)
        psim = 0.
        if len(pitem_property_list) > 0:
            for i in pitem_property_list:
                psim += cos_sim(item_property_list, i)
            psim /= len(pitem_property_list)
        
        #统计category命中率
        hit_cate_rate = 0.
        hit_cate_sim = 1.
        if len(item_category_list) > 1:
            if len(item_category_list) == 2 and item_category_list[1] in pitem_category_list_prop:
                hit_cate_rate += 1
                hit_cate_sim *= (1+cos_sim(pitem_category_list_prop[item_category_list[1]], item_property_list))
            if len(item_category_list) == 3 and item_category_list[2] in pitem_category_list_prop:
                hit_cate_rate += 1
                hit_cate_sim *= (1+cos_sim(pitem_category_list_prop[item_category_list[2]], item_property_list))
            
            hit_cate_rate /= (len(item_category_list) - 1)
            
            
        
        
        predict_richness = len(set(pitem_category_list))
        item_property_richness = len(set(item_property_list))
        return [csim, psim, predict_richness, item_property_richness, hit_cate_rate, hit_cate_sim]
    
        
        
    print("processing predict_category_property ...")
#     dfX['predict_category_property'] = dfX['predict_category_property'].str.split(';').map(lambda x: [i.split(":")[0] for i in x]).map(filter_unless_cate)
    
    dfX['tmp'] = dfX['item_category_list']+"|"+dfX['item_property_list']+"|"+dfX['predict_category_property']
    dfX['tmp'] = dfX['tmp'].map(inner_product_recall_items)
    
    dfX['category_sim'] = dfX['tmp'].map(lambda x: x[0])
    dfX['property_sim'] = dfX['tmp'].map(lambda x: x[1])
    dfX['predict_richness'] =  dfX['tmp'].map(lambda x: x[2])
    dfX['item_property_richness'] = dfX['tmp'].map(lambda x: x[3])
    dfX['hit_cate_cnt'] = dfX['tmp'].map(lambda x: x[4])
    dfX['hit_cate_sim'] = dfX['tmp'].map(lambda x: x[5])
    del dfX["tmp"]
    
    print("processing item_property_list ...")
    
    print("remove dup props ...")
    dfX['item_property_list'] = dfX['item_property_list'].str.split(';').map(filter_bad_data)
    
    #分层统计prop交易率
    print("stat trading rate ...")
    tmp_df = dfX[["is_trade", 'item_property_list', "day"]]
    for day in xrange(8):
        s1 = stat_prop_trade_rate()
        stat_df = None
        if day == 0:
            stat_df = tmp_df[tmp_df["day"]==0]
        else:
            stat_df = tmp_df[tmp_df["day"]<day]
        
        print("set %s => %s" % (stat_df["day"].unique(), day))
        stat_df.apply(s1, axis=1)
        
        print("porp to layer cnt")
        p = prop_layer_cnt(s1.get_m())
        dfX.loc[dfX["day"]==day, "tmp"] = dfX['item_property_list'].map(p)
    
    for i in xrange(len(list(dfX[0:1]['tmp'])[0])):
        print("process item_property_list_%s ..." % i)
        dfX['item_property_list_%s' % i] = dfX['tmp'].map(lambda x: x[i])
    del dfX["tmp"]
    
    #     print("prop to vector ...")
#     p2v = prop2vec(10, list(dfX["item_property_list"]))
#     dfX["item_property_list_vec"] = dfX["item_property_list"].map(p2v)
#     for i in xrange(10):
#         print("process item_property_list_f%s ..." % i)
#         dfX["item_property_list_f%s" % i] = dfX["item_property_list_vec"].map(lambda x: x[i])
#     del dfX["item_property_list_vec"]
    
#     dfX['item_property_list1'] = dfX['item_property_list'].map(s1.get_topbutton_cnt_filter(80,80,500))
#     dfX['item_property_list2'] = dfX['item_property_list'].map(s1.get_topbutton_cnt_filter(50,50,2000))
#     dfX['item_property_list3'] = dfX['item_property_list'].map(s1.get_topbutton_cnt_filter(20,15,5000))
#     dfX['item_property_list_sub'] = dfX[['item_property_list1', 'item_property_list2', 'item_property_list3']].apply(lambda x: list(set(x["item_property_list1"]+x["item_property_list2"]+x["item_property_list3"])), axis=1)
    
#     del dfX['item_property_list1']
#     del dfX['item_property_list2']
#     del dfX['item_property_list3']
    
    print("processing item_category_list ...")
    dfX['item_category_list'] = dfX['item_category_list'].str.split(';')
    
    print "generating item_category_1, item_category_2 ..."
#     dfX['item_category_list01'] = dfX['item_category_list'].map(lambda x:x[0] if x != None and len(x) > 0 else None)
    dfX['item_category_1'] = dfX['item_category_list'].map(lambda x:int(x[1]) if x != None and len(x) > 1 else None)
    dfX['item_category_2'] = dfX['item_category_list'].map(lambda x:int(x[2]) if x != None and len(x) > 2 else None)
    
    return dfX



# aaa = process_complex_types(train_df.copy(), get_icl_map(train_df), get_ipl_map(train_df))

train_df_prev = process_complex_types(train_df_prev.copy(), get_icl_map(train_df_prev), get_ipl_map(train_df_prev))

# process_complex_types(train_df_prev.copy(), None, None)

# train_df_prev["item_category_1"] = train_df_prev["item_category_1"].astype('int')
# train_df_prev["item_category_2"] = train_df_prev["item_category_2"].astype('int')

get_icl_map ... 
get_ipl_map ... 
processing predict_category_property ...
processing item_property_list ...
remove dup props ...
stat trading rate ...
set [0] => 0
porp to layer cnt
set [0] => 1
porp to layer cnt
set [0 1] => 2
porp to layer cnt
set [0 1 2] => 3
porp to layer cnt
set [0 3 1 2] => 4
porp to layer cnt
set [0 3 1 2 4] => 5
porp to layer cnt
set [0 3 1 2 4 5] => 6
porp to layer cnt
set [0 3 1 2 4 5 6] => 7
porp to layer cnt
process item_property_list_0 ...
process item_property_list_1 ...
process item_property_list_2 ...
process item_property_list_3 ...
process item_property_list_4 ...
process item_property_list_5 ...
process item_property_list_6 ...
process item_property_list_7 ...
process item_property_list_8 ...
process item_property_list_9 ...
process item_property_list_10 ...
process item_property_list_11 ...
process item_property_list_12 ...
process item_property_list_13 ...
process item_property_list_14 ...
process item_property_list_15 ...
process item_property_li

In [6]:
train_df_prev[['item_property_list_0', 'item_property_list_1','item_property_list_2','item_property_list_3','item_property_list_4','item_property_list_5','item_property_list_6','item_property_list_7','item_property_list_8','item_property_list_9','item_property_list_10','item_property_list_11','item_property_list_12','item_property_list_13','item_property_list_14','item_property_list_15','item_property_list_16','item_property_list_17','item_property_list_18','item_property_list_19']].describe()

,item_property_list_0,item_property_list_1,item_property_list_2,item_property_list_3,item_property_list_4,item_property_list_5,item_property_list_6,item_property_list_7,item_property_list_8,item_property_list_9,item_property_list_10,item_property_list_11,item_property_list_12,item_property_list_13,item_property_list_14,item_property_list_15,item_property_list_16,item_property_list_17,item_property_list_18,item_property_list_19
count,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000
mean,0.192220,0.193608,8.355742,13.606867,0.032620,0.003474,0.191420,0.619473,0.591363,0.057812,0.287038,0.322359,0.916535,1.027993,0.373125,1.341388,0.746911,1.094687,1.363180,2.811794
std,0.579059,0.689246,6.326289,8.512245,0.180295,0.059791,0.759947,2.254591,1.272502,0.258051,1.871843,1.687188,2.951310,2.617253,1.323974,4.581186,2.890635,3.336093,3.533861,5.522848
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,4.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,6.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,10.000000,21.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000
max,2.000000,7.000000,48.000000,40.000000,2.000000,2.000000,7.000000,39.000000,22.000000,3.000000,28.000000,26.000000,41.000000,39.000000,17.000000,48.000000,39.000000,41.000000,39.000000,48.000000


## 重复列特征

In [7]:
def generateColDupByDay(df, cols, gaps, dateCol='day', verbose=True):
    """
    生成新特征, 根据日期, 把特定列的去重状态排列出来

    Parameters
    ----------------
    df : data
    col : 指定列
    gap : 相隔多少天
    dateCol : 日期列, 'day', 以自然数代表天数

    offline_df = generateColDupByDay(offline_df, ['item_id', 'shop_id'], [1,2,3])
    print(offline_df.item_id_dup_g_3)
    print(offline_df.shop_id_dup_g_1)

    Returns
    ----------------
    new DataFrame with new features
    """   
    tmp = df.copy()
    new_features = []
    for col in cols:
        for gap in gaps:
            new_col = col + '_dup_gg_' + str(gap) 
            if verbose:
                print('setting col ' + new_col)
            new_features.append(new_col)
            setColDupByDay(tmp, col, gap, new_col, dateCol)
    return tmp, new_features

def setColDupByDay(df, col, gap, newCol, dateCol='day'):

    # 新特征列
    if newCol in df:
        return df
    
    days = len(df[dateCol].unique())

    for today in range(0, days+1):
 
        # 获取 今天以来, 今天 这两种索引
        index_alldays = np.logical_and(df[dateCol].values <= today, df[dateCol].values > (today - gap - 1))
        index_today = (df[dateCol].values == today)

        if len(df[index_today]) == 0 or len(df[index_alldays]) == 0:
            break

        # 计算今天以来col的重复状态, 并且把今天的状态拿出来set到新列
        df.loc[index_today, newCol] = df.loc[index_alldays, col].duplicated()


    # 把两种状态转换成0, 1
    le = preprocessing.LabelEncoder()
    df[newCol] = le.fit_transform(df[newCol].fillna('nodata').astype('str'))

    return df

dup_feat = ['item_id', 'item_brand_id', 'shop_id', 'user_id']
train_df_prev, dup_features = generateColDupByDay(train_df_prev, dup_feat, list(range(1, 8)))

setting col item_id_dup_gg_1
setting col item_id_dup_gg_2
setting col item_id_dup_gg_3
setting col item_id_dup_gg_4
setting col item_id_dup_gg_5
setting col item_id_dup_gg_6
setting col item_id_dup_gg_7
setting col item_brand_id_dup_gg_1
setting col item_brand_id_dup_gg_2
setting col item_brand_id_dup_gg_3
setting col item_brand_id_dup_gg_4
setting col item_brand_id_dup_gg_5
setting col item_brand_id_dup_gg_6
setting col item_brand_id_dup_gg_7
setting col shop_id_dup_gg_1
setting col shop_id_dup_gg_2
setting col shop_id_dup_gg_3
setting col shop_id_dup_gg_4
setting col shop_id_dup_gg_5
setting col shop_id_dup_gg_6
setting col shop_id_dup_gg_7
setting col user_id_dup_gg_1
setting col user_id_dup_gg_2
setting col user_id_dup_gg_3
setting col user_id_dup_gg_4
setting col user_id_dup_gg_5
setting col user_id_dup_gg_6
setting col user_id_dup_gg_7


## 统计一个用户重复看某个商品的次数+一个用户重复看某类商品的次数

In [8]:
class set_review_cnt:
    def __init__(self, key1, key2, cnt_k1_k2_review):
        self.key1 = key1
        self.key2 = key2
        self.tmp = collections.defaultdict(int)
        self.cnt_k1_k2_review = cnt_k1_k2_review
    def __call__(self, x):
        val1,val2 = x[self.key1], x[self.key2]
        vk = (val1, val2)
        if val2 < 0:
            return np.nan 
        if val2 == None:
            return np.nan

        cnt = self.cnt_k1_k2_review[vk]
        if cnt == 0:
            return 0
        else:
            
            k = "%s_%s" % (val1, val2)
            ret = self.tmp[k]
            self.cnt_k1_k2_review[vk] -= 1
            self.tmp[k] += 1
            return ret
        
def use_feautre_review_cnt(df, feature, new_feature):
    
    cnt_feature_review = df[["user_id", feature, "instance_id"]].groupby(["user_id", feature])['instance_id'].count().to_dict() 
    f = set_review_cnt("user_id", feature, cnt_feature_review)
    
    df = df.sort_values(by="context_timestamp")
    df[new_feature] = df[["user_id", feature]].apply(f, axis=1)
    
    return df

    

    
tmp_df = train_df_prev.copy()

features = [
    ["item_id", "item_review_cnt"],
    ["item_category_1", "cate_review_cnt"],
#     ["item_category_2", "cate2_review_cnt"],
    ["item_brand_id", "brand_review_cnt"],
    ["item_price_level", "price_review_cnt"],
    ["item_sales_level", "sales_review_cnt"],
    ["item_pv_level", "pv_review_cnt"],
    ["item_collected_level", "collected_review_cnt"],
    ["item_city_id", "city_review_cnt"],
    ["shop_id", "shop_review_cnt"],
    ["shop_review_num_level", "shop_review_lv_review_cnt"],
    ["shop_star_level", "shop_star_lv_review_cnt"]
]

show_features = []
for f in features:
    print "caling %s into %s ..." % (f[0], f[1])
    tmp_df = use_feautre_review_cnt(tmp_df, f[0], f[1])
    show_features.append(f[1])
    

caling item_id into item_review_cnt ...
caling item_category_1 into cate_review_cnt ...
caling item_brand_id into brand_review_cnt ...
caling item_price_level into price_review_cnt ...
caling item_sales_level into sales_review_cnt ...
caling item_pv_level into pv_review_cnt ...
caling item_collected_level into collected_review_cnt ...
caling item_city_id into city_review_cnt ...
caling shop_id into shop_review_cnt ...
caling shop_review_num_level into shop_review_lv_review_cnt ...
caling shop_star_level into shop_star_lv_review_cnt ...


In [9]:
train_df_prev = tmp_df
# for f in show_features:
#     print "\n", f
#     for v in train_df_prev[f].unique():
#         print v, len(train_df_prev[train_df_prev[f] == v])
#     print '-' * 30
#     print ""
cnt_feautre_review = show_features
train_df_prev[show_features].describe()


,item_review_cnt,cate_review_cnt,brand_review_cnt,price_review_cnt,sales_review_cnt,pv_review_cnt,collected_review_cnt,city_review_cnt,shop_review_cnt,shop_review_lv_review_cnt,shop_star_lv_review_cnt
count,496509.000000,496509.000000,496018.000000,496509.000000,495561.000000,496509.000000,496509.000000,496226.000000,496509.000000,496509.000000,496509.000000
mean,0.086780,1.109394,0.198612,0.831550,0.319499,0.367798,0.318697,0.598862,0.126040,0.321219,0.347110
std,0.365856,2.142329,0.647292,1.697039,0.800378,0.878178,0.796520,1.459132,0.467344,0.807286,0.855506
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,15.000000,52.000000,20.000000,49.000000,27.000000,26.000000,24.000000,52.000000,18.000000,27.000000,27.000000


# 转化属性增加trade_rate + trade_pv（按前一天）

In [10]:
# 统计信息附加

def calcTVTransform(df, key, key_y, filter_src, filter_dst, smoothing = -1, mean0=None):
    """
    Examples:
    -----------
    is_trade是是否交易列, 0为无交易, 1为有交易, 对item_city_id计算交易信息
    r = calcTVTransform(total_df, 'item_city_id', 'is_trade', (total_df['day'] == 5), (total_df['day'] == 6))

    Returns:
    -----------
    r
    r['exp'] # item_city_id列的is_trade概率
    r['sum'] # item_city_id列的is_trade和
    r['cnt'] # item_city_id列的is_trade总数
    """
    if mean0 is None:
        #计算目标的平均值做平缓用
        mean0 = df.ix[filter_src, key_y].mean()
        # print("mean0:", mean0)
    
    #取出key的所有值
    df['_key1'] = df[key].astype('category').values.codes
    
    
    #取出用于计算的源（后面聚合掉就没有顺序可言了）
    df_key1_y = df.ix[filter_src, ['_key1', key_y]]
    
    #根据key的取值去聚合key_y的总数和总和，用户计算rate和count
    grp1 = df_key1_y.groupby(['_key1'])
    sum1 = grp1[key_y].aggregate(np.sum)
    cnt1 = grp1[key_y].aggregate(np.size)
    
    vn_sum = 'sum_' + key
    vn_cnt = 'cnt_' + key
    
    #取出dst（带序列）的所有key
    v_codes = df.ix[filter_dst, '_key1']
    
    #得到_sum,_cnt，按dst的序列
    _sum = sum1[v_codes].values
    _cnt = cnt1[v_codes].values
    _cnt[np.isnan(_sum)] = 0    
    _sum[np.isnan(_sum)] = 0
    
    r = {}
    if smoothing > 0:
        r['exp'] = (_sum + smoothing * mean0)/(_cnt + smoothing)
    else:
        r['exp'] = (_sum)/(_cnt)
    r['sum'] = _sum
    r['cnt'] = _cnt
    return r


In [11]:
#计算前一天的交易率set到下一天，第0天用回自己
# tmp = train_df.copy()
tmp = train_df_prev

add_count = True
# window = 2

exp = "exp_d_"
cnt = "cnt_d_"


exp_numerical = {}
cnt_numerical = {}

In [12]:

k = "day"
exp_k = "exp_d_day"
for day in xrange(0,7):
    cal_day = day
    set_day = day

    print("cal %s trade_rate cnt %s set to %s" % (k,cal_day, set_day))

    #start_d - day(不含day)用于计算，结果赋值到day上
    days1 = (tmp.day.values == cal_day)
    days2 = (tmp.day.values == set_day)
    ret = calcTVTransform(tmp, k, 'is_trade', days1, days2, 0)

    tmp.loc[tmp.day.values == day, exp_k] = ret["exp"]

    exp_numerical[exp_k]=1

tmp.loc[tmp.day.values == 7, exp_k] = 0.0169

tmp[[exp_k, "day"]]

cal day trade_rate cnt 0 set to 0


/Users/yuhua/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


cal day trade_rate cnt 1 set to 1
cal day trade_rate cnt 2 set to 2
cal day trade_rate cnt 3 set to 3
cal day trade_rate cnt 4 set to 4
cal day trade_rate cnt 5 set to 5
cal day trade_rate cnt 6 set to 6


,exp_d_day,day
3844,0.0000,0
24046,0.0000,0
59329,0.0000,0
70767,0.0000,0
76650,0.0000,0
27196,0.0000,0
57458,0.0000,0
59196,0.0000,0
24479,0.0000,0
23301,0.0000,0


In [13]:
# 计算前一天的交易率set到下一天，第0天用回自己
# %run ../util/time_utils.py

from datetime import datetime, timedelta
import pandas as pd
from sklearn import preprocessing
import numpy as np



def generateTradeRateByDate(tmp, cols, gap, exp_numerical, cnt_numerical, colSmoothing=None, verbose=True, glbSmoothing=10, glbMean0=0.5):
    add_count = False

    total_days = 8

    exp = "exp_d_"
    cnt = "cnt_d_"

    #此处应该处理按天为梯度的数据
    for k in cols:
        exp_k = exp+k
        cnt_k = cnt+k
        
        alpha, beta = 0, 0
        if colSmoothing and k in colSmoothing:
            alpha = colSmoothing[k][0]
            beta = colSmoothing[k][1]
            if verbose:
                print('activating smoothing: alpha %s, beta %s' % (alpha, beta))

        for day in range(0, 8):

            cal_day = max(day - 1, 0)
            set_day = day

            
            #start_d - day(不含day)用于计算，结果赋值到day上
            days1 = np.logical_and(tmp.day.values <= cal_day, tmp.day.values > (cal_day - gap))
            days2 = (tmp.day.values == set_day)
            ret = calcTVTransform(tmp, k, 'is_trade', days1, days2, smoothing=glbSmoothing, mean0=glbMean0)
             
            # 内建平滑
            tmp.loc[tmp.day.values == day, exp_k] = ret['exp'] 

            # 外部平滑
            # tmp.loc[tmp.day.values == day, exp_k] = (ret['sum']+alpha)/(ret['cnt']+alpha+beta)

            exp_numerical[exp_k] = 1
            tmp.loc[tmp.day.values == day, cnt_k] = ret["cnt"]
            cnt_numerical[cnt_k] = 1
            if verbose:
                # print(" %s trade_rate cal_day %s set to day %s" % (k, ))
                print('(%s -> %s) spawn cols by %s' % (tmp[days1]["day"].unique(), set_day, k))





trade_rela = ['item_price_level',
'item_sales_level',
'item_collected_level',
'item_pv_level',
'user_gender_id',
'user_age_level',
'user_occupation_id',
'user_star_level',
'context_page_id',
'shop_review_num_level',
'shop_star_level',
         

"item_brand_id",
"item_city_id",
"item_id",
"shop_id",
"user_id",
         ]
# trade_rela = ['item_city_id', 'item_id', 'item_brand_id', 'shop_id', 'user_id']

# colSm = {}
# for col in trade_rela:
#     colSm[col] = [15*(mean0) , 15]
generateTradeRateByDate(tmp, trade_rela, 7,  exp_numerical, cnt_numerical, None, verbose=True, glbSmoothing=200, glbMean0=0.05)



([0] -> 0) spawn cols by item_price_level
([0] -> 1) spawn cols by item_price_level
([0 1] -> 2) spawn cols by item_price_level


/Users/yuhua/anaconda2/lib/python2.7/site-packages/pandas/core/series.py:696: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


([0 1 2] -> 3) spawn cols by item_price_level
([0 1 2 3] -> 4) spawn cols by item_price_level
([0 1 2 3 4] -> 5) spawn cols by item_price_level
([0 1 2 3 4 5] -> 6) spawn cols by item_price_level
([0 1 2 3 4 5 6] -> 7) spawn cols by item_price_level
([0] -> 0) spawn cols by item_sales_level
([0] -> 1) spawn cols by item_sales_level
([0 1] -> 2) spawn cols by item_sales_level
([0 1 2] -> 3) spawn cols by item_sales_level
([0 1 2 3] -> 4) spawn cols by item_sales_level
([0 1 2 3 4] -> 5) spawn cols by item_sales_level
([0 1 2 3 4 5] -> 6) spawn cols by item_sales_level
([0 1 2 3 4 5 6] -> 7) spawn cols by item_sales_level
([0] -> 0) spawn cols by item_collected_level
([0] -> 1) spawn cols by item_collected_level
([0 1] -> 2) spawn cols by item_collected_level
([0 1 2] -> 3) spawn cols by item_collected_level
([0 1 2 3] -> 4) spawn cols by item_collected_level
([0 1 2 3 4] -> 5) spawn cols by item_collected_level
([0 1 2 3 4 5] -> 6) spawn cols by item_collected_level
([0 1 2 3 4 5 6] -> 

In [14]:
mean_h_features = {}

def generateTradeMeanHourByDate(tmp, cols, gap, exp_numerical, colSmoothing=None, verbose=True, glbSmoothing=10, glbMean0=0.5):
    add_count = False

    total_days = 8

    exp = "mean_hour_"

    #此处应该处理按天为梯度的数据
    for k in cols:
        exp_k = exp+k
        
        alpha, beta = 0, 0
        if colSmoothing and k in colSmoothing:
            alpha = colSmoothing[k][0]
            beta = colSmoothing[k][1]
            if verbose:
                print('activating smoothing: alpha %s, beta %s' % (alpha, beta))
        
        cal_df = tmp.copy()
#         cal_df.loc[cal_df["is_trade"] == 0, "hour"] = -1
        for day in range(0, 8):

            cal_day = max(day - 1, 0)
            set_day = day

            
            #start_d - day(不含day)用于计算，结果赋值到day上
            days1 = np.logical_and(tmp.day.values <= cal_day, tmp.day.values > (cal_day - gap))
            days1 = np.logical_and(cal_df.is_trade.values == 1, days1)
            days2 = (tmp.day.values == set_day)
            ret = calcTVTransform(cal_df[[k, 'hour']], k, 'hour', days1, days2)#, smoothing=glbSmoothing, mean0=glbMean0)
            
            # 内建平滑
            tmp.loc[tmp.day.values == day, exp_k] = ret['exp'] 
            mean_h_features[exp_k] = 1
            if verbose:
                # print(" %s trade_rate cal_day %s set to day %s" % (k, ))
                print('%s (%s -> %s) spawn cols by %s' % (exp_k, tmp[days1]["day"].unique(), set_day, k))





trade_rela = [
'item_price_level',
'item_sales_level',
'item_collected_level',
'item_pv_level',
'user_gender_id',
'user_age_level',
'user_occupation_id',
'user_star_level',
'context_page_id',
'shop_review_num_level',
'shop_star_level',
         

"item_brand_id",
"item_city_id",
"item_id",
"shop_id",
"user_id",
         ]

generateTradeMeanHourByDate(tmp, trade_rela, 7,  mean_h_features, None, verbose=True, glbSmoothing=10, glbMean0=None)

/Users/yuhua/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/Users/yuhua/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/yuhua/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in divide


mean_hour_item_price_level ([0] -> 0) spawn cols by item_price_level
mean_hour_item_price_level ([0] -> 1) spawn cols by item_price_level
mean_hour_item_price_level ([0 1] -> 2) spawn cols by item_price_level
mean_hour_item_price_level ([0 1 2] -> 3) spawn cols by item_price_level
mean_hour_item_price_level ([0 1 2 3] -> 4) spawn cols by item_price_level
mean_hour_item_price_level ([0 1 2 3 4] -> 5) spawn cols by item_price_level
mean_hour_item_price_level ([0 1 2 3 4 5] -> 6) spawn cols by item_price_level
mean_hour_item_price_level ([0 1 2 3 4 5 6] -> 7) spawn cols by item_price_level
mean_hour_item_sales_level ([0] -> 0) spawn cols by item_sales_level
mean_hour_item_sales_level ([0] -> 1) spawn cols by item_sales_level
mean_hour_item_sales_level ([0 1] -> 2) spawn cols by item_sales_level
mean_hour_item_sales_level ([0 1 2] -> 3) spawn cols by item_sales_level
mean_hour_item_sales_level ([0 1 2 3] -> 4) spawn cols by item_sales_level
mean_hour_item_sales_level ([0 1 2 3 4] -> 5) spa

mean_hour_shop_id ([0 1 2] -> 3) spawn cols by shop_id
mean_hour_shop_id ([0 1 2 3] -> 4) spawn cols by shop_id
mean_hour_shop_id ([0 1 2 3 4] -> 5) spawn cols by shop_id
mean_hour_shop_id ([0 1 2 3 4 5] -> 6) spawn cols by shop_id
mean_hour_shop_id ([0 1 2 3 4 5 6] -> 7) spawn cols by shop_id
mean_hour_user_id ([0] -> 0) spawn cols by user_id
mean_hour_user_id ([0] -> 1) spawn cols by user_id
mean_hour_user_id ([0 1] -> 2) spawn cols by user_id
mean_hour_user_id ([0 1 2] -> 3) spawn cols by user_id
mean_hour_user_id ([0 1 2 3] -> 4) spawn cols by user_id
mean_hour_user_id ([0 1 2 3 4] -> 5) spawn cols by user_id
mean_hour_user_id ([0 1 2 3 4 5] -> 6) spawn cols by user_id
mean_hour_user_id ([0 1 2 3 4 5 6] -> 7) spawn cols by user_id


In [15]:
exp_numerical = exp_numerical.keys()
cnt_numerical = cnt_numerical.keys()

print(exp_numerical)
print(cnt_numerical)



['exp_d_user_age_level', 'exp_d_item_pv_level', 'exp_d_shop_review_num_level', 'exp_d_item_sales_level', 'exp_d_user_occupation_id', 'exp_d_item_city_id', 'exp_d_item_brand_id', 'exp_d_user_star_level', 'exp_d_item_price_level', 'exp_d_day', 'exp_d_item_id', 'exp_d_shop_star_level', 'exp_d_item_collected_level', 'exp_d_user_gender_id', 'exp_d_context_page_id', 'exp_d_shop_id', 'exp_d_user_id']
['cnt_d_shop_id', 'cnt_d_shop_review_num_level', 'cnt_d_context_page_id', 'cnt_d_user_id', 'cnt_d_item_collected_level', 'cnt_d_user_age_level', 'cnt_d_item_sales_level', 'cnt_d_item_price_level', 'cnt_d_item_id', 'cnt_d_shop_star_level', 'cnt_d_item_city_id', 'cnt_d_item_brand_id', 'cnt_d_user_occupation_id', 'cnt_d_user_star_level', 'cnt_d_user_gender_id', 'cnt_d_item_pv_level']


In [33]:
print "RECOVER"
train_df = train_df_prev.copy().sort_index()

RECOVER


In [34]:
d = train_df[["shop_id", "item_id", "instance_id"]].groupby(["shop_id", "item_id"])['instance_id'].count().to_dict()
m = {}
for i in list(train_df["shop_id"].unique()):
    m[i] = 0
for k,v in d.items():
    m[k[0]] += 1

train_df["shop_item_cnt"] = train_df["shop_id"].map(lambda x: m[x])

In [35]:
m = collections.defaultdict(dict)
def get_user_t_series(x):
    m[x["user_id"]][x["instance_id"]] = x["context_timestamp"]
    return 0

def get_last_access_minute(x):
    arr = m[x["user_id"]]
    max_idx = len(arr)-1
    target_idx = -1
    for idx,v in enumerate(arr):
        if v[0] == x["instance_id"]:
            target_idx = idx
    
    if target_idx == max_idx:
        return np.nan
    else:
        return int((arr[target_idx][1] - arr[target_idx+1][1]) / 60)

def get_next_access_minute(x):
    arr = m[x["user_id"]]
    max_idx = len(arr)-1
    target_idx = -1
    for idx,v in enumerate(arr):
        if v[0] == x["instance_id"]:
            target_idx = idx
    
    if target_idx - 1 < 0:
        return np.nan
    else:
        return int((arr[target_idx-1][1] - arr[target_idx][1]) / 60)

train_df[["user_id", "instance_id", "context_timestamp"]].apply(get_user_t_series, axis=1)

for k,v in m.items():
    m[k] = sorted(v.items(),key = lambda x:x[1], reverse = True)

train_df["next_access_min"] = train_df[["user_id", "instance_id"]].apply(get_next_access_minute, axis=1)
train_df["last_access_min"] = train_df[["user_id", "instance_id"]].apply(get_last_access_minute, axis=1)


# 标记训练用的列

In [36]:
#标记训练数据

import copy
target="is_trade"
                          
                          
                        
categorical=[  
                'item_price_level',
                'item_sales_level',
                'item_collected_level',
                'item_pv_level',
                'user_gender_id',
                'user_age_level',
                'user_occupation_id',
                'user_star_level',
                'context_page_id',
                'shop_review_num_level',
                'shop_star_level',
                "weekday",
                "hour",
    
                'item_category_1',
                'item_category_2',
                
                "item_brand_id",
                "item_city_id",
                "item_id",
                "shop_id",
    
            ]+cnt_feautre_review+dup_features+cnt_numerical

numerical=[     'shop_review_positive_rate',
                'shop_score_service',
                'shop_score_delivery',
                'shop_score_description',
#                 'predict_category_property',
                'predict_richness',
                'item_property_richness'
                   
#               ]
          ]+exp_numerical

listype = [
    'item_property_list', 
#     'item_category_list'
]

                    
            

cols_tool = filter_on_cols(target, categorical, numerical, listype)

In [37]:
train_df[cols_tool.get_raw_numerical_cols()].describe()

,shop_review_positive_rate,shop_score_service,shop_score_delivery,shop_score_description,predict_richness,item_property_richness,exp_d_user_age_level,exp_d_item_pv_level,exp_d_shop_review_num_level,exp_d_item_sales_level,exp_d_user_occupation_id,exp_d_item_city_id,exp_d_item_brand_id,exp_d_user_star_level,exp_d_item_price_level,exp_d_day,exp_d_item_id,exp_d_shop_star_level,exp_d_item_collected_level,exp_d_user_gender_id,exp_d_context_page_id,exp_d_shop_id,exp_d_user_id
count,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000
mean,0.994833,0.971131,0.970495,0.974870,4.810191,34.235279,0.020020,0.020218,0.020329,0.020329,0.019859,0.021107,0.029319,0.020077,0.019760,0.000625,0.039220,0.020262,0.020280,0.019841,0.020428,0.033680,0.049781
std,0.011875,0.023607,0.023567,0.024823,1.795871,10.100147,0.002941,0.002100,0.003383,0.005478,0.001379,0.007537,0.011857,0.002656,0.007961,0.003190,0.010449,0.003435,0.002758,0.001507,0.001729,0.012354,0.000758
min,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,6.000000,0.014028,0.015750,0.010426,0.009329,0.014056,0.013177,0.006555,0.011009,0.009681,0.000000,0.008734,0.011039,0.013442,0.013433,0.013631,0.006352,0.039062
25%,0.992780,0.966360,0.965677,0.969282,3.000000,28.000000,0.018510,0.018682,0.017890,0.016309,0.019004,0.014385,0.021695,0.019898,0.015256,0.000000,0.031933,0.017902,0.018932,0.019368,0.019782,0.025090,0.049751
50%,1.000000,0.972347,0.971579,0.978493,5.000000,33.000000,0.019321,0.020024,0.020041,0.019232,0.019520,0.018362,0.027197,0.020546,0.016334,0.000000,0.041096,0.019316,0.019474,0.019500,0.020423,0.033898,0.050000
75%,1.000000,0.977822,0.976966,0.983626,6.000000,38.000000,0.020798,0.020753,0.022343,0.022669,0.020987,0.025167,0.037736,0.021083,0.024191,0.000000,0.047170,0.022955,0.021700,0.019642,0.020652,0.043307,0.050000
max,1.000000,1.000000,1.000000,1.000000,14.000000,99.000000,0.036415,0.053140,0.052885,0.041485,0.036415,0.063636,0.101156,0.040650,0.053659,0.016900,0.100826,0.055300,0.042194,0.024855,0.031315,0.099673,0.063725


# 平滑处理连续型数据最后对复杂类型做onehot编码

In [38]:
#数据的预处理
#double系列(例如好评率)的数据存在一个问题，没有考虑到评论量，所以乘了一个评论level，类似 好评率*评论数(分桶了) = 好评量(分桶了)
#fillempty把-1设置成-0.01，配合mov2pos使用
#mov2pos把有<0的数据都集体-min，如果一个数据是0-1，有-1出现的时候，上面设置成了-0.01，所以这列数据真实是-0.01 - 1,然后我会集体-(-0.01)
#norm用了最大值最小值norm
%run FeatureProcess.py
from sklearn import preprocessing


def layering(df, key, by_key):
    uniqv = df[by_key].unique()
    keys = []
    for v in uniqv:
        indexer = (df[by_key] == v)
        new_key = "*%sLAY*_%s" % (str(v), key)
        df.loc[indexer, new_key] = df[indexer][key]
        keys.append(new_key)
    return df, keys





featProc = FeatureProcess(target=target, categorical=categorical, numerical=numerical, listype = listype)

train_df = featProc.fillempty(train_df, -0.01)
# train_df = featProc.mov2pos(train_df)
train_df = featProc.mov2mean(train_df)
train_df = featProc.norm(train_df)
train_df[cols_tool.get_raw_numerical_cols()].describe()


,shop_review_positive_rate,shop_score_service,shop_score_delivery,shop_score_description,predict_richness,item_property_richness,exp_d_user_age_level,exp_d_item_pv_level,exp_d_shop_review_num_level,exp_d_item_sales_level,exp_d_user_occupation_id,exp_d_item_city_id,exp_d_item_brand_id,exp_d_user_star_level,exp_d_item_price_level,exp_d_day,exp_d_item_id,exp_d_shop_star_level,exp_d_item_collected_level,exp_d_user_gender_id,exp_d_context_page_id,exp_d_shop_id,exp_d_user_id
count,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000
mean,0.982014,0.865794,0.827843,0.883320,0.293092,0.303605,0.267659,0.119485,0.233240,0.342073,0.259543,0.157153,0.240639,0.305906,0.229182,0.037000,0.331042,0.208377,0.237811,0.561071,0.384352,0.292834,0.434603
std,0.032251,0.043904,0.054594,0.056408,0.138144,0.108604,0.131368,0.056164,0.079672,0.170362,0.061670,0.149374,0.125334,0.089598,0.181034,0.188763,0.113457,0.077607,0.095920,0.131921,0.097796,0.132386,0.030741
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.974731,0.842328,0.798142,0.856064,0.153846,0.236559,0.200197,0.078413,0.175780,0.217072,0.221291,0.023924,0.160042,0.299890,0.126775,0.000000,0.251910,0.155075,0.190937,0.519670,0.347854,0.200783,0.433392
50%,1.000000,0.870375,0.832816,0.899184,0.307692,0.290323,0.236407,0.114309,0.226442,0.307969,0.244387,0.102759,0.218199,0.321756,0.151286,0.000000,0.351409,0.187020,0.209775,0.531145,0.384064,0.295175,0.443478
75%,1.000000,0.896039,0.864508,0.923249,0.384615,0.344086,0.302405,0.133791,0.280662,0.414842,0.309986,0.237620,0.329606,0.339844,0.329935,0.000000,0.417364,0.269224,0.287189,0.543606,0.397036,0.395997,0.443478
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# 定义模型函数

In [39]:
%run ffmyh.py
        
def train_and_test_lgb(train_df, test_df, categorical_feature, res_df):
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    
    model = lgb.LGBMClassifier(num_leaves=63, max_depth=7, n_estimators=80, n_jobs=5)
    model.fit(X_train, y_train, feature_name=list(X_train.columns),
            categorical_feature=categorical_feature)
    
    ft_w = pd.DataFrame(model.feature_importances_, columns=["weights"], index = list(X_train.columns))
    print ft_w
    
    train_lls = log_loss(y_train,model.predict_proba(X_train))
    test_lls = log_loss(y_test,model.predict_proba(X_test))
    
    if type(res_df) != type(None):
        del res_df["is_trade"]
        res_df = res_df.reset_index(drop=True)
        instance_id_list = res_df[["instance_id"]]
        del res_df["instance_id"]
        predicted_score = pd.DataFrame(model.predict_proba(res_df))
        
        instance_id_list["predicted_score"] = predicted_score[1]
        instance_id_list.to_csv("xgboost_res.csv",index=False,sep=' ')
        
    
    del train_df
    del test_df
    
    del X_train
    del y_train
    del X_test
    del y_test
    
    return train_lls, test_lls

def train_and_test_xgboost(train_df, test_df, res_df, print_importance=False):    
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    
#     model = xgboost.XGBClassifier(reg_lambda=1.5,learning_rate=0.1,reg_alpha=0,nthread=7,n_estimators=100,max_depth=10)
#     model = xgboost.XGBClassifier(subsample=0.9,colsample_bytree=0.9,n_estimators=91,max_depth=5,nthread=7, gamma=6.6)
    model = xgboost.XGBClassifier(n_jobs=7,max_depth=5,n_estimators=91,min_child_weight=5, gamma=6.6)

#     model = xgboost.XGBClassifier(subsample=0.9,colsample_bytree=0.9,n_estimators=50,max_depth=3,nthread=1)
    model.fit(X_train, y_train)
    
    if print_importance:
        fig, ax = plt.subplots(figsize=(12,18))
        xgboost.plot_importance(model,ax=ax)
        plt.show()

    train_lls = log_loss(y_train,model.predict_proba(X_train))
    test_lls = log_loss(y_test,model.predict_proba(X_test))
    
    if type(res_df) != type(None):
        del res_df["is_trade"]
        res_df = res_df.reset_index(drop=True)
        instance_id_list = res_df[["instance_id"]]
        del res_df["instance_id"]
        predicted_score = pd.DataFrame(model.predict_proba(res_df))
        
        instance_id_list["predicted_score"] = predicted_score[1]
        instance_id_list.to_csv("xgboost_res.csv",index=False,sep=' ')
        
    
    del train_df
    del test_df
    
    del X_train
    del y_train
    del X_test
    del y_test
    
    return train_lls, test_lls

def train_and_test_lr(train_df, test_df, res_df):    
    X_train = train_df.copy().fillna(0)
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy().fillna(0)
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    
    
    model = LogisticRegression(C=1.0, n_jobs=7,tol=1e-6, max_iter=2000)
    model.fit(X_train.values, y_train)
#     for idx, val in enumerate(list(X_train.columns)):
#         print("%s=%s" %(val,list(model.coef_[0])[idx]))


    train_lls = log_loss(y_train,model.predict_proba(X_train))
    test_lls = log_loss(y_test,model.predict_proba(X_test))
    
    if type(res_df) != type(None):
        res_df = res_df.copy().fillna(0)
        del res_df["is_trade"]
        res_df = res_df.reset_index(drop=True)
        instance_id_list = res_df[["instance_id"]]
        del res_df["instance_id"]
        predicted_score = pd.DataFrame(model.predict_proba(res_df))
        
        instance_id_list["predicted_score"] = predicted_score[1]
        instance_id_list.to_csv("lr_res.csv",index=False,sep=' ')
    
    del train_df
    del test_df
    
    del X_train
    del y_train
    del X_test
    del y_test
    
    return train_lls, test_lls

def train_and_test_randomforest(train_df, test_df, res_df):    
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
#     rf = RandomForestClassifier(n_estimators=32, max_depth=40, min_samples_split=100, min_samples_leaf=10,  criterion='entropy',
#                         max_features=8, verbose = 1,  bootstrap=False, n_jobs=10)
#     RandomForestClassifier(criterion='entropy',n_estimators=100,n_jobs=15)

    model = RandomForestClassifier(random_state=9,n_jobs=7, n_estimators=100, max_depth=10, min_samples_split=100, min_samples_leaf=10,  criterion='entropy', max_features=8, verbose = 1,  bootstrap=False)
    model.fit(X_train, y_train)
    
    ft_w = pd.DataFrame(model.feature_importances_, columns=["weights"], index = list(X_train.columns))
    print ft_w.sort_values("weights", ascending=False)
#     fig, ax = plt.subplots(figsize=(12,18))
#     xgboost.plot_importance(model,ax=ax)
#     plt.show()
    
    train_lls = log_loss(y_train,model.predict_proba(X_train))
    test_lls = log_loss(y_test,model.predict_proba(X_test))
    
    
    if type(res_df) != type(None):
        del res_df["is_trade"]
        res_df = res_df.reset_index(drop=True)
        instance_id_list = res_df[["instance_id"]]
        del res_df["instance_id"]
        predicted_score = pd.DataFrame(model.predict_proba(res_df))
        
        instance_id_list["predicted_score"] = predicted_score[1]
        instance_id_list.to_csv("randomforest_res.csv",index=False,sep=' ')
        
        
   
    del train_df
    del test_df
    
    del X_train
    del y_train
    del X_test
    del y_test
    
    return train_lls, test_lls

def train_and_test_gbdt(train_df, test_df, res_df):
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    model = GradientBoostingClassifier(n_estimators=100, max_depth=10, min_samples_split=100, min_samples_leaf=10, max_features=4)
    model.fit(X_train, y_train)
    
    train_lls = log_loss(y_train,model.predict_proba(X_train))
    test_lls = log_loss(y_test,model.predict_proba(X_test))
    
    if type(res_df) != type(None):
        del res_df["is_trade"]
        res_df = res_df.reset_index(drop=True)
        instance_id_list = res_df[["instance_id"]]
        del res_df["instance_id"]
        predicted_score = pd.DataFrame(model.predict_proba(res_df))
        
        instance_id_list["predicted_score"] = predicted_score[1]
        instance_id_list.to_csv("gbdt_res.csv",index=False,sep=' ')
        
    del train_df
    del test_df
    
    del X_train
    del y_train
    del X_test
    del y_test
    
    return train_lls, test_lls




def train_and_test_FFM(train_df, test_df, res_df):
    global cols_tool
    
    model = FFM(target=cols_tool.target, categorical=cols_tool.categorical, numerical=cols_tool.numerical, listype=cols_tool.listype,
                                        reg_param = 0.0004,
                                        k = 4,
                                        iter_max = 500,
                                        learing_rate = 0.1,
                                        threads = 7,
                                        auto_stop = False,
                                        quiet = False,
                                        no_norm = False)


    
    model.fit(train_df, test_df)
    
    train_lls = log_loss(train_df["is_trade"],model.predict_proba(train_df))
    test_lls = log_loss(test_df["is_trade"],model.predict_proba(test_df))
    
    
    if type(res_df) != type(None):
        del res_df["is_trade"]
        res_df = res_df.reset_index(drop=True)
        instance_id_list = res_df[["instance_id"]]
        del res_df["instance_id"]
        predicted_score = pd.DataFrame(model.predict_proba(res_df))
        
        instance_id_list["predicted_score"] = predicted_score[1]
        instance_id_list.to_csv("ffm.csv",index=False,sep=' ')
        
    
    return train_lls, test_lls


def select_best_features(df, precent=0.8):
    from sklearn.feature_selection import SelectKBest,chi2 
    df = df.copy().replace(-1, 0).fillna(0)
    
    for u in df.columns:
        df[df[u] == -1] = 0

    X_train = df.copy()
    del X_train['is_trade']
    y_train = df['is_trade']
    
    c = int(len(X_train.columns) * precent)
    skb = SelectKBest(chi2,k=c)
    skb.fit_transform(X_train,y_train)
    
    old_fea = list(X_train.columns)
    new_fea = []
    for idx, b in enumerate(list(skb.get_support())):
        if b:
            new_fea.append(old_fea[idx])
        else:
            print("Filter feature:%s" % old_fea[idx])
    return new_fea

# 开搞！

In [40]:

def dim_reduce_topn(df, dim, n, v=1):
    df2 = pd.DataFrame(df[dim].value_counts()/len(df))
    m = dict(df2.nlargest(n, dim)[dim])
    df[dim] = df[dim].map(lambda x: x if x in m else v)

    return df

def dim_reduce_rate(df, dim, rate, v=1):
    df2 = pd.DataFrame(df[dim].value_counts()/len(df))
    m = dict(df2[df2[dim] > rate][dim])
    df[dim] = df[dim].map(lambda x: x if x in m else v)
    return df

def dim_reduce_cnt(df, dim, cnt, v=1):
    df2 = pd.DataFrame(df[dim].value_counts())
    m = dict(df2[df2[dim] > cnt][dim])
    df[dim] = df[dim].map(lambda x: x if x in m else v)
    return df

def bucketizer_linear(df, key, bins, cover=False):
    nkey="*BUK*_"+key
    df[nkey] = np.nan
    idxer = df[key].values < bins[0]
    print idxer
    b = 1
    df.loc[idxer, nkey] = b
    print "x < %s : %s" % (bins[0], b)
    b += 1
    start = bins[0]
    for end in bins[1:]:
        idxer = np.logical_and(start <= df[key].values, df[key].values < end)
        df.loc[idxer, nkey] = b
        
        print "%s <= x < %s : %s" % (start, end, b)
        b+=1
        
        start = end
    
    print "%s <= x : %s" % (bins[-1], b)
    idxer = bins[-1] <= df[key].values
    df.loc[idxer, nkey] = b
    if cover:
        df[key] = df[nkey]
        del df[nkey]
#         df.drop(nkey)
    return df

def bucketizer_cnt(df, key, bins, cover=False):
    bins -= 1
    l = sorted(list(df[key].unique()))
    
    step = (max(l) - min(l)) / bins
    curr = step
    arr_bins = [curr]
    curr += step
    
    while curr < max(l):
        arr_bins.append(curr)
        curr += step
    
    print key, "has", len(l), "values projection to", len(arr_bins)+1, "buckets"
    bucketizer_linear(df, key, arr_bins, cover)


def bucketizer_rate(df, key, bins, cover=False):
    l = sorted(list(df[key].unique()))
    bins = (len(l) * bins) - 1
    
    step = (max(l) - min(l)) / bins
    curr = step
    arr_bins = [curr]
    curr += step
    
    while curr < max(l):
        arr_bins.append(curr)
        curr += step
    
    print key, "has", len(l), "values projection to", len(arr_bins)+1, "buckets"
    bucketizer_linear(df, key, arr_bins, cover)
    

    
# tmp_train_df["prop_len"] = tmp_train_df["item_property_list"].map(lambda x: len(x))
# tmp_train_df[["prop_len"]].describe()

In [46]:
#这块一般不怎么需要改了，拿最后一天做验证
#注意下面验证版和测试版，测试版是我们自己玩的，验证版是扔线上的，通过注释不同的跑就行了


# %run BaseFrame.py
% run ../util/time_series_split.py
%run FeatureProcess.py
total_train_df = train_df.copy()#.fillna(-1)


#reduce
print len(dim_reduce_rate(total_train_df, "shop_id", 0.01)["shop_id"].unique())
print len(dim_reduce_rate(total_train_df, "item_city_id", 0.01)["item_city_id"].unique())
print len(dim_reduce_rate(total_train_df, "item_brand_id", 0.01)["item_brand_id"].unique())
print len(dim_reduce_topn(total_train_df, "item_id", 30)["item_id"].unique())


dim_reduce_topn(total_train_df, "item_review_cnt", 3, 666)
dim_reduce_topn(total_train_df, "cate_review_cnt", 8, 666)
dim_reduce_topn(total_train_df, "brand_review_cnt", 3, 666)


bucketizer_linear(total_train_df, "price_review_cnt", [1,2,3,4,5,6,7,8,9,10,12], True)
bucketizer_linear(total_train_df, "sales_review_cnt", [1,2,3,4,5,6], True)
dim_reduce_topn(total_train_df, "pv_review_cnt", 6, 666)
dim_reduce_topn(total_train_df, "collected_review_cnt", 6, 666)
bucketizer_linear(total_train_df, "city_review_cnt", [1,2,3,4,5,6,7,8,9,12], True)

for k in total_train_df.columns:
    if k.find("cnt_d_") == 0:
        uniql = len(total_train_df[k].unique())
        if uniql < 10:
            continue
        elif uniql < 30:
            bucketizer_cnt(total_train_df, k, 5, True)
        else:
            bucketizer_cnt(total_train_df, k, 10, True)


cols_tool.fit(total_train_df)

categorical_one_hot_cols = cols_tool.get_onehoted_cols("categorical")
listype_one_hot_cols = cols_tool.get_onehoted_cols("listype")

index_col = "instance_id"


categorical_cols = cols_tool.get_raw_categorical_cols()
listype_cols = cols_tool.get_raw_listype_cols()
numerical_cols = cols_tool.get_raw_numerical_cols()
target_col = cols_tool.get_raw_target_col()


print("org dataset = %s" % len(total_train_df))
print(total_train_df["day"].value_counts())


print(total_train_df["day"].value_counts())

#测试版
valid_df = total_train_df.loc[total_train_df["day"]==6]
sub_train_df = total_train_df.loc[total_train_df["day"]<6]
# sub_train_df = sub_train_df.loc[sub_train_df["day"]>0]
res_df=valid_df.copy()

# #线上验证版
# valid_df = total_train_df.loc[total_train_df["day"]==6]
# res_df = total_train_df.loc[total_train_df["day"]==7]
# sub_train_df = total_train_df.loc[total_train_df["day"]<=6]

# print("res dataset = %s" % len(res_df))
# print("valid dateset = %s" % len(valid_df))
# print("train dateset = %s" % len(sub_train_df))

# print "X1",len(train_df_prev.columns)
# print "X2",len(set(train_df_prev.columns))

# on_cols = [target_col]+categorical_cols+numerical_cols
# on_cols = ["is_trade",'item_id', 'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level', 'context_page_id', 'shop_id', 'shop_review_num_level', 'shop_review_positive_rate', 'shop_star_level', 'shop_score_service', 'shop_score_delivery', 'shop_score_description', 'day', 'weekday', 'hour', 'item_id_dup_gg_1', 'item_id_dup_gg_2', 'item_id_dup_gg_3', 'item_id_dup_gg_4', 'item_id_dup_gg_5', 'item_id_dup_gg_6', 'item_id_dup_gg_7', 'item_brand_id_dup_gg_1', 'item_brand_id_dup_gg_2', 'item_brand_id_dup_gg_3', 'item_brand_id_dup_gg_4', 'item_brand_id_dup_gg_5', 'item_brand_id_dup_gg_6', 'item_brand_id_dup_gg_7', 'shop_id_dup_gg_1', 'shop_id_dup_gg_2', 'shop_id_dup_gg_3', 'shop_id_dup_gg_4', 'shop_id_dup_gg_5', 'shop_id_dup_gg_6', 'shop_id_dup_gg_7', 'user_id_dup_gg_1', 'user_id_dup_gg_2', 'user_id_dup_gg_3', 'user_id_dup_gg_4', 'user_id_dup_gg_5', 'user_id_dup_gg_6', 'user_id_dup_gg_7', '_key1', 'exp_d_item_city_id', 'cnt_d_item_city_id', 'exp_d_item_id', 'cnt_d_item_id', 'exp_d_item_brand_id', 'cnt_d_item_brand_id', 'exp_d_shop_id', 'cnt_d_shop_id', 'exp_d_user_id', 'cnt_d_user_id', 'category_sim', 'property_sim', 'predict_richness', 'item_property_richness', 'hit_cate_cnt', 'hit_cate_sim', 'item_category_1', 'item_category_2', 'item_review_cnt', 'cate_review_cnt' 
# #            , 'brand_review_cnt'
# # , 'price_review_cnt'
# # , 'sales_review_cnt'
# # , 'pv_review_cnt'
# # , 'collected_review_cnt'
# # , 'city_review_cnt'
#           ]

on_cols = ['predict_richness',
'user_id_dup_gg_7',
'user_id_dup_gg_6',
'user_id_dup_gg_5',
'cnt_d_item_pv_level',
'item_id_dup_gg_7',
'item_id_dup_gg_6',
'item_id_dup_gg_5',
'item_id_dup_gg_4',
'item_id_dup_gg_3',
'item_id_dup_gg_2',
'exp_d_user_star_level',
'pv_review_cnt',
'sales_review_cnt',
'item_brand_id_dup_gg_7',
'item_brand_id_dup_gg_6',
'item_brand_id_dup_gg_5',
'item_brand_id_dup_gg_4',
'item_brand_id_dup_gg_3',
'item_brand_id_dup_gg_2',
'item_brand_id_dup_gg_1',
'user_age_level',
'shop_id_dup_gg_6',
'shop_id_dup_gg_7',
'shop_id_dup_gg_4',
'shop_id_dup_gg_5',
'shop_id_dup_gg_2',
'shop_id_dup_gg_3',
'shop_id_dup_gg_1',
'exp_d_day',
'context_page_id',
'shop_review_positive_rate',
'cnt_d_item_price_level',
'cnt_d_user_occupation_id',
'shop_id',
'exp_d_item_pv_level',
'item_price_level',
'item_id_dup_gg_1',
'brand_review_cnt',
'item_id',
'exp_d_shop_star_level',
'cnt_d_item_id',
'exp_d_shop_id',
'hour',
'exp_d_user_occupation_id',
'user_gender_id',
'cnt_d_context_page_id',
'price_review_cnt',
'user_id_dup_gg_2',
'user_id_dup_gg_4',
'item_sales_level',
'exp_d_user_age_level',
'cnt_d_user_age_level',
'exp_d_item_sales_level',
'city_review_cnt',
'cnt_d_shop_review_num_level',
'item_city_id',
'item_category_2',
'item_review_cnt',
'shop_score_description',
'exp_d_context_page_id',
'cnt_d_user_gender_id',
'cnt_d_user_id']+[target_col]+["last_access_min","shop_item_cnt","next_access_min"]+['shop_review_cnt','shop_review_lv_review_cnt','shop_star_lv_review_cnt']
#+['item_property_list_2','item_property_list_3','item_property_list_5','item_property_list_8','item_property_list_9','item_property_list_11','item_property_list_15','item_property_list_17']
#ff = ['item_property_list_0','item_property_list_1','item_property_list_2','item_property_list_3','item_property_list_4','item_property_list_5','item_property_list_6','item_property_list_7','item_property_list_8','item_property_list_9','item_property_list_10','item_property_list_11','item_property_list_12','item_property_list_13','item_property_list_14','item_property_list_15','item_property_list_16','item_property_list_17','item_property_list_18','item_property_list_19']

#+['last_access_min']+mean_h_features.keys()

#+['shop_item_cnt']
#["item_property_list_l1p", "item_property_list_l1n", "item_property_list_l2p", "item_property_list_l2n", "item_property_list_l3p", "item_property_list_l3n"]

#+['item_property_list_f0','item_property_list_f1','item_property_list_f2','item_property_list_f3','item_property_list_f4','item_property_list_f5','item_property_list_f6','item_property_list_f7','item_property_list_f8','item_property_list_f9']



train_lls, test_lls = train_and_test_xgboost(sub_train_df[on_cols], valid_df[on_cols], res_df[on_cols+[index_col]], False)
print("XGBoost %s dimension TrainLogLoss = %s | TestLogLoss = %s" % (len(sub_train_df[on_cols].columns), train_lls, test_lls))


# last_lls = test_lls
# for i in ff:
#     on_cols = list(set(on_cols+[i]))
#     # on_cols = select_best_features(total_train_df[on_cols], 0.9)+[target_col]


# #     print "Using Features ..."
# #     print on_cols
# #     print "total", len(on_cols)

#     train_lls, test_lls = train_and_test_xgboost(sub_train_df[on_cols], valid_df[on_cols], None, False)
#     print("XGBoost %s dimension TrainLogLoss = %s | TestLogLoss = %s" % (len(sub_train_df[on_cols].columns), train_lls, test_lls))
    
#     print "%s diff = %s" % (i, test_lls-last_lls)
    
#     last_lls = test_lls



# train_lls, test_lls = train_and_test_randomforest(sub_train_df[on_cols], valid_df[on_cols], res_df[on_cols + [index_col]])
# print("RandomForest %s dimension TrainLogLoss = %s | TestLogLoss = %s" % (len(sub_train_df[on_cols].columns), train_lls, test_lls))



# def feat_filter(arr, feats):
#     ret = []
#     for i in arr:
#         if i not in feats:
#             continue
#         else:
#             ret.append(i)
#     return ret

# ffmFeatProc = FeatureProcess(target=target, categorical=feat_filter(categorical, on_cols), numerical=feat_filter(numerical, on_cols), listype=[])#listype = listype)
# ffmFeatProc.fit(total_train_df)
# ffmFeatProc.toFFMData(sub_train_df, "one_hot-train-sflr.txt")
# ffmFeatProc.toFFMData(valid_df, "one_hot-valid-sflr.txt")

# model = FFM(ffmFeatProc)
# model.fit_schema()
# print "\nPARSE_RES\n"
# model.parse_model("./tmp.txt", "./ffm/sflr.model")


# train_lls, test_lls = train_and_test_lgb(sub_train_df[on_cols], valid_df[on_cols], cols_tool["categorical"], res_df[on_cols + [index_col]])
# print("LightGBM %s dimension TrainLogLoss = %s | TestLogLoss = %s" % (len(sub_train_df[on_cols].columns), train_lls, test_lls))




################特征分析
# def run_on_check_feature(sub_train_df, valid_df, f1, f2, target_col, res_train, res_test):
    
#     use_fea = f1+[target_col]
    
#     #计算出第一组base_score
#     train_lls, test_lls = train_and_test_xgboost(sub_train_df[use_fea], valid_df[use_fea], None)
#     base_score_train = train_lls
#     base_score_test = test_lls
    
#     #一个个feature加进去训练
#     for f in f2:
#         use_fea = f1+[f,target_col]
        
# #         print use_fea
#         print "TestFeature:"
#         print "[%s]" % f
    
#         train_lls, test_lls = train_and_test_xgboost(sub_train_df[use_fea], valid_df[use_fea], None)
        
#         #计算出加了这个feature的diff
#         diff_train_lls = train_lls - base_score_train 
#         diff_test_lls = test_lls - base_score_test
        
#         print "[BASE] test = %s | train = %s" % (base_score_test, base_score_train)
#         print "[THIS] test = %s | train = %s" % (test_lls, train_lls)
#         print "[DIFF] test = %s | train = %s" % (diff_test_lls, diff_train_lls)
        
#         print ""
#         print "-" * 60
#         print ""
        
#         #记录下来
#         res_train[f].append(diff_train_lls)
#         res_test[f].append(diff_test_lls)
        
#         #这次的score就更新为base
#         base_score_train = train_lls
#         base_score_test = test_lls
            
#         #这次检测完的feature也加进去用
#         f1.append(f)

# import random
# on_cols = categorical_cols+numerical_cols

# # on_cols = on_cols[:4]
# #特征挑选
# res_train = collections.defaultdict(list)
# res_test = collections.defaultdict(list)
# for i in xrange(5):
#     random.shuffle(on_cols)
    
#     #把特征一分为二，然后50%用来验证，50%为基线，后面再交换
#     f1 = on_cols[len(on_cols)/2:]
#     f2 = on_cols[:len(on_cols)/2]
    
#     print "F1 Round"
#     run_on_check_feature(sub_train_df, valid_df, f1[:], f2[:], target_col, res_train, res_test)
    
#     print "\nF2 Round"
#     run_on_check_feature(sub_train_df, valid_df, f2[:], f1[:], target_col, res_train, res_test)
    
#     print "Whole Round", i, "\n\n"
    
################特征分析

5
12
7
31
[ True  True  True ... False  True  True]
x < 1 : 1
1 <= x < 2 : 2
2 <= x < 3 : 3
3 <= x < 4 : 4
4 <= x < 5 : 5
5 <= x < 6 : 6
6 <= x < 7 : 7
7 <= x < 8 : 8
8 <= x < 9 : 9
9 <= x < 10 : 10
10 <= x < 12 : 11
12 <= x : 12


/Users/yuhua/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in less


[ True  True  True ...  True False  True]
x < 1 : 1
1 <= x < 2 : 2
2 <= x < 3 : 3
3 <= x < 4 : 4
4 <= x < 5 : 5
5 <= x < 6 : 6
6 <= x : 7


/Users/yuhua/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in greater_equal
/Users/yuhua/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in less
/Users/yuhua/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in greater_equal


[ True  True  True ... False  True  True]
x < 1 : 1
1 <= x < 2 : 2
2 <= x < 3 : 3
3 <= x < 4 : 4
4 <= x < 5 : 5
5 <= x < 6 : 6
6 <= x < 7 : 7
7 <= x < 8 : 8
8 <= x < 9 : 9
9 <= x < 12 : 10
12 <= x : 11
cnt_d_item_price_level has 76 values projection to 9 buckets
[ True  True  True ... False False False]
x < 23253.88888888889 : 1
23253.88888888889 <= x < 46507.77777777778 : 2
46507.77777777778 <= x < 69761.66666666667 : 3
69761.66666666667 <= x < 93015.55555555556 : 4
93015.55555555556 <= x < 116269.44444444445 : 5
116269.44444444445 <= x < 139523.33333333334 : 6
139523.33333333334 <= x < 162777.22222222225 : 7
162777.22222222225 <= x < 186031.11111111112 : 8
186031.11111111112 <= x : 9
cnt_d_item_sales_level has 126 values projection to 10 buckets
[ True  True  True ... False False False]
x < 9651.777777777777 : 1
9651.777777777777 <= x < 19303.555555555555 : 2
19303.555555555555 <= x < 28955.333333333332 : 3
28955.333333333332 <= x < 38607.11111111111 : 4
38607.11111111111 <= x < 4825

/Users/yuhua/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


XGBoost 70 dimension TrainLogLoss = 0.08419867718113834 | TestLogLoss = 0.07962600775545073


- | XGboost | 增减 | LR |增减
- | :-: | :-: | :-: | :-:
原始特征 | 0.0829350458274 | / | 0.0829428179401 | /
原始特征+时间 | 0.0828004994761 | +++ | 0.082869098617 | +
同上+按天计算交易率 | 0.0828218434014 | - | 0.0827914564667 | ++
同上+predict_prop_cate相似度 | 0.082778758021 | ++ | 0.082798919764 | --
同上+cate prop丰富度 | 0.0828356414143 | -- | 0.0828009736655 | -
同上+cate/prop list OneHot | 0.0828356414143 | / | 0.0827979171146 | +